In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from tqdm import tqdm
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *
%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/Downloads/cobrapy-0.5.11/cobra/__init__.py
/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f5aad734198>

In [3]:
with open('./me_models/iYO844b.pickle', 'rb') as f:
    me = pickle.load(f)
with open('./me_models/final/iJL1678b.pickle', 'rb') as f:
    ecome = pickle.load(f)

In [4]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-6)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 213.324819 seconds
Finished substituting S,lb,ub in 9.539917 seconds
Finished makeME_LP in 1.105094 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 9.584070 seconds
Finished makeME_LP in 1.046729 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 9.221459 seconds
Finished makeME_LP in 1.043934 seconds
Getting MINOS parameters from ME_NLP...
3 0.0625 0.0625 0.125 0.0625 optimal
Finished substituting S,lb,ub in 9.449980 seconds
Finished makeME_LP in 1.071222 seconds
Getting MINOS parameters from ME_NLP...
4 0.09375 0.09375 0.125 0.09375 optimal
Finished substituting S,lb,ub in 9.382561 seconds
Finished makeME_LP in 1.063486 seconds
Getting MINOS parameters from ME_NLP...
5 0.09375 0.09375 0.109375 0.109375 1
Finished substituting S,lb,ub in 9.186386 seconds
Finished makeME_LP in 1.061670 seconds
Gettin

In [5]:
solve_me_model(ecome, 1., min_mu = .1, precision=1e-6, using_soplex=False)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 421.665303 seconds
Finished substituting S,lb,ub in 17.033310 seconds
Finished makeME_LP in 1.882164 seconds
Getting MINOS parameters from ME_NLP...
1 0.5 0.5 1.0 0.5 optimal
Finished substituting S,lb,ub in 17.917648 seconds
Finished makeME_LP in 1.873168 seconds
Getting MINOS parameters from ME_NLP...
2 0.75 0.75 1.0 0.75 optimal
Finished substituting S,lb,ub in 17.361421 seconds
Finished makeME_LP in 1.875130 seconds
Getting MINOS parameters from ME_NLP...
3 0.75 0.75 0.875 0.875 1
Finished substituting S,lb,ub in 17.140175 seconds
Finished makeME_LP in 1.862176 seconds
Getting MINOS parameters from ME_NLP...
4 0.8125 0.8125 0.875 0.8125 optimal
Finished substituting S,lb,ub in 16.846597 seconds
Finished makeME_LP in 1.856605 seconds
Getting MINOS parameters from ME_NLP...
5 0.84375 0.84375 0.875 0.84375 optimal
Finished substituting S,lb,ub in 16.886299 seconds
Finished makeME_LP in 1.850010 seconds
Get

In [6]:
df = exchange_single_model(ecome)
df.to_csv('temp_result.csv')

In [5]:
with open('./me_models/solution.pickle', 'wb') as solution:
    pickle.dump(me,solution)

In [8]:
with open('./me_models/ecoli_solution.pickle', 'wb') as solution:
    pickle.dump(ecome,solution)

# Postprocessing

In [9]:
with open('./me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)
with open('./me_models/ecoli_solution.pickle', 'rb') as solution:
    ecome = pickle.load(solution)

In [10]:
# Exchange
df_m = exchange_single_model(bsub)
df = exchange_single_model(me)
df.join(df_m['flux'],lsuffix='_me',rsuffix='_m')

,name,reaction,lower_bound,upper_bound,flux_me,flux_m
id,,,,,,
EX_co2_e,,co2_e <=>,-999999.0,999999.0,6.045741e+00,5.980676
EX_glc__D_e,,glc__D_e <=>,-1.7,999999.0,-1.699999e+00,-1.700000
EX_h2o_e,,h2o_e <=>,-999999.0,999999.0,8.426608e+00,7.869819
EX_h_e,,h_e <=>,-999999.0,999999.0,7.867541e-01,0.767665
EX_k_e,,k_e <=>,-999999.0,999999.0,-1.809908e-07,-0.083321
EX_mg2_e,,mg2_e <=>,-999999.0,999999.0,-1.350856e-03,-0.011999
EX_nh4_e,,nh4_e <=>,-5.0,999999.0,-9.459037e-01,-0.944964
EX_o2_e,,o2_e <--,-999999.0,0.0,-5.796162e+00,-5.705899
EX_pi_e,,pi_e <=>,-5.0,999999.0,-1.068200e-01,-0.181053


In [11]:
me.reactions.get_by_id('EX_trp__L_e').upper_bound

999999.0

In [12]:
# Solution summary
summary_df = solution_summary(me)
summary_m_df = solution_summary(bsub)
summary_df.to_csv('./solution_summary.csv')

100%|██████████| 1250/1250 [00:02<00:00, 514.41it/s]


In [13]:
summary_df = solution_summary(me)

100%|██████████| 6190/6190 [00:43<00:00, 143.68it/s]


## Compare M/ME with ecoli

In [14]:
_,rxn_id_dict = homogenize_reactions(model=bsub,ref_model=eco)

100%|██████████| 1250/1250 [00:00<00:00, 3337.25it/s]


In [15]:
# M - ME comparison of metabolic fluxes in Bacillus
flux_dict = me.get_metabolic_flux()
me_flux_df = pd.DataFrame.from_dict({'flux':flux_dict}).rename(index=rxn_id_dict)
comparison_df = summary_m_df.join(me_flux_df,lsuffix='_m',rsuffix='_me')
comparison_df[abs(comparison_df.flux_m)>0]

,lb,ub,flux_m,formula,flux_me
2S6HCCi,0,999999,3.13791e-05,akg_c + h_c + ichor_c --> 2shchc_c + co2_c + p...,0.000027
ACACT1r,-999999,999999,0.00330023,2.0 accoa_c <=> aacoa_c + coa_c,0.002848
ACACT2r,-999999,999999,0.00330023,accoa_c + btcoa_c <=> 3ohcoa_c + coa_c,0.002848
ACACT3r,-999999,999999,0.00330023,accoa_c + hxcoa_c <=> 3oocoa_c + coa_c,0.002848
ACACT4r,-999999,999999,0.00330023,accoa_c + occoa_c <=> 3odcoa_c + coa_c,0.002848
...,...,...,...,...,...
UMPK,0,999999,0.0383322,atp_c + ump_c --> adp_c + udp_c,0.055243
URIDK2r_copy1,0,999999,999999,atp_c + dump_c --> adp_c + dudp_c,0.000000
URIDK2r_copy2,-999999,999999,-999999,atp_c + dump_c <=> adp_c + dudp_c,-0.004862
VALDHr,-999999,999999,999998,h2o_c + nad_c + val__L_c <=> 3mob_c + h_c + na...,0.000000


In [16]:
# M - ME comparison of metabolic fluxes in E. coli
summary_df_ecoli = solution_summary(ecome)
summary_m_df_ecoli = solution_summary(eco)

flux_dict_ecoli = ecome.get_metabolic_flux()
comparison_df_ecoli = summary_m_df_ecoli.join(\
                    pd.DataFrame.from_dict({'flux':flux_dict_ecoli}),lsuffix='_m',rsuffix='_me')

  0%|          | 0/2583 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
summary_m_df[summary_m_df.flux > 0]

In [ ]:
me.metabolites.ribosome.formula_weight/1000

In [ ]:
import matplotlib.pyplot as plt
threshold = 100
temp_df = comparison_df[abs(comparison_df.flux_m)<threshold]
temp_df_ecoli = comparison_df_ecoli[abs(comparison_df_ecoli.flux_m)<threshold]

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.scatter(temp_df['flux_m'],temp_df['flux_me'])
plt.xlabel('M simulation')
plt.ylabel('ME simulation')
plt.title('Bacillus')

plt.subplot(1,2,2)
plt.scatter(temp_df_ecoli['flux_m'],temp_df_ecoli['flux_me'])
plt.xlabel('M simulation')
plt.ylabel('ME simulation')
plt.title('E. coli')


In [ ]:
# Store results to visualize in Escher
comparison_df['flux_m'].to_csv('fluxdist_m_bsub.csv',header=False)
comparison_df['flux_me'].to_csv('fluxdist_me_bsub.csv',header=False)

comparison_df_ecoli['flux_m'].to_csv('fluxdist_m_ecoli.csv',header=False)
comparison_df_ecoli['flux_me'].to_csv('fluxdist_me_ecoli.csv',header=False)

## Gene essentiality

In [1]:
from __future__ import print_function, division, absolute_import

from copy import copy
# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
import numpy as np

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

from cobrame.core.component import (Constraint, ProcessedProtein, Complex,
                                    TranslatedGene, TranscribedGene, Metabolite, Ribosome,
                                    GenericComponent,GenerictRNA,RNAP)

print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)

In [3]:
with open('./me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)
with open('./me_models/ecoli_solution.pickle', 'rb') as solution:
    ecome = pickle.load(solution)

In [4]:
NP = 16

In [ ]:
GE_dict_m = gene_essentiality(bsub, model_type = 'm',  lim = 0.01, NP=NP)

Number of processors:  16
BSU16730 0 -1.6845470178651136e-10
BSU07840 0 -2.9606060061953983e-06
BSU30820 e -1.0
BSU16740 0 -1.6845470178651136e-10
BSU06310 0 -3.015210791159218e-10
BSU39860 0 0.0
BSU03040 0 -2.3118499904190636e-10
BSU38830 0 0.0
BSU03900 0 6.264465396683538e-11
BSU38560 0 0.0
BSU19310 0 0.0
BSU40110 0 0.0
BSU32830 0 -0.0009092498653858589
BSU24170 0 -0.00082955738714317
BSU22440 e -1.0000000000000207
BSU29210 e -1.0000000000000207
BSU24340 e -1.0000000000000207
BSU29200 e -1.0000000000000207
BSU13900 - -0.01336478034198814
BSU07700 0 -1.021307980261297e-10
BSU13910 - -0.01336478034198814
BSU11210 e -1.0
BSU11200 e -1.0
BSU28300 0 0.0
BSU24350 e -1.0000000000000207
BSU36010 0 0.0
BSU28310 0 0.0
BSU29470 0 -0.0015819159855061501
BSU36000 0 -9.532122761147827e-05
BSU18260 0 0.0
BSU32820 0 0.0
BSU37170 0 0.0
BSU24150 0 0.0
BSU04520 0 0.0
BSU15350 0 -5.097167372735683e-10
BSU18000 - -0.15208314442876789
BSU11220 e -1.0
BSU04620 0 -2.7521082560425924e-10
BSU19230 0 -5.801282

In [ ]:
GE_dict_ecoli_m = gene_essentiality(eco, model_type = 'm',  lim = 0.01, NP=NP)

In [ ]:
GE_dict_me = gene_essentiality(me, model_type = 'me',  lim = 0.01, NP=NP, initial_f=0)

In [ ]:
GE_dict_ecoli_me = gene_essentiality(ecome, model_type = 'me',  lim = 0.01, NP=NP, initial_f=0)

In [ ]:
# Create one dict
def unify_dict(GE_dict_m,GE_dict_me):
    GE_dict = {}
    for key in GE_dict_me.keys():
        GE_dict[key] = {}
        try:
            GE_dict[key]['m'] = GE_dict_m[key]
        except:
            GE_dict[key]['m'] = None
        GE_dict[key]['me'] = GE_dict_me[key]

    for key in GE_dict_m.keys():
        if key not in GE_dict.keys():
            GE_dict[key] = {}
            GE_dict[key]['m'] = GE_dict_m[key]
            try:
                GE_dict[key]['me'] = GE_dict_me[key]
            except:
                GE_dict[key]['me'] = None        
    return GE_dict

In [ ]:
GE_dict = unify_dict(GE_dict_m,GE_dict_me)

In [ ]:
GE_dict_ecoli = unify_dict(GE_dict_ecoli_m,GE_dict_ecoli_me)
GE_dict_ecoli

In [12]:
with open("./me_models/GE_dict.pickle", "wb") as outfile:
    pickle.dump(GE_dict, outfile)
with open("./me_models/GE_dict_ecoli.pickle", "wb") as outfile:
    pickle.dump(GE_dict_ecoli, outfile)

In [29]:
with open('./me_models/GE_dict.pickle', 'rb') as outfile:
    GE_dict = pickle.load(outfile)
with open('./me_models/GE_dict_ecoli.pickle', 'rb') as outfile:
    GE_dict_ecoli = pickle.load(outfile)

In [5]:
for key in GE_dict.keys():
    if GE_dict[key]['m'] != GE_dict[key]['me']:
        print(key, GE_dict[key])

BSU00410 {'m': None, 'me': '0'}
BSU00380 {'m': '0', 'me': 'e'}
BSU00130 {'m': None, 'me': 'e'}
BSU00550 {'m': None, 'me': 'e'}
BSU00810 {'m': None, 'me': '0'}
BSU00980 {'m': None, 'me': '0'}
BSU00820 {'m': None, 'me': 'e'}
BSU01070 {'m': None, 'me': 'e'}
BSU01000 {'m': None, 'me': 'e'}
BSU00940 {'m': None, 'me': 'e'}
BSU01080 {'m': None, 'me': 'e'}
BSU01010 {'m': None, 'me': 'e'}
BSU01100 {'m': None, 'me': 'e'}
BSU01120 {'m': None, 'me': 'e'}
BSU01130 {'m': None, 'me': 'e'}
BSU01110 {'m': None, 'me': 'e'}
BSU01150 {'m': None, 'me': 'e'}
BSU01200 {'m': None, 'me': 'e'}
BSU01220 {'m': None, 'me': 'e'}
BSU01300 {'m': None, 'me': 'e'}
BSU01230 {'m': None, 'me': 'e'}
BSU01310 {'m': None, 'me': 'e'}
BSU01260 {'m': None, 'me': 'e'}
BSU01250 {'m': None, 'me': 'e'}
BSU01290 {'m': None, 'me': 'e'}
BSU01340 {'m': None, 'me': 'e'}
BSU01320 {'m': None, 'me': 'e'}
BSU01350 {'m': None, 'me': 'e'}
BSU01330 {'m': None, 'me': 'e'}
BSU01380 {'m': None, 'me': 'e'}
BSU01360 {'m': None, 'me': 'e'}
BSU01410 

In [4]:
GE_dict

{'BSU00110': {'m': '0', 'me': '0'},
 'BSU00180': {'m': '0', 'me': '0'},
 'BSU00140': {'m': '0', 'me': '0'},
 'BSU00410': {'m': None, 'me': '0'},
 'BSU00270': {'m': '0', 'me': '0'},
 'BSU00120': {'m': '0', 'me': '0'},
 'BSU00730': {'m': '0', 'me': '0'},
 'BSU00150': {'m': '0', 'me': '0'},
 'BSU00090': {'m': '0', 'me': '0'},
 'BSU00680': {'m': '0', 'me': '0'},
 'BSU00760': {'m': 'e', 'me': 'e'},
 'BSU00500': {'m': 'e', 'me': 'e'},
 'BSU00460': {'m': 'e', 'me': 'e'},
 'BSU00380': {'m': '0', 'me': 'e'},
 'BSU00740': {'m': 'e', 'me': 'e'},
 'BSU00750': {'m': 'e', 'me': 'e'},
 'BSU00130': {'m': None, 'me': 'e'},
 'BSU00280': {'m': 'e', 'me': 'e'},
 'BSU00550': {'m': None, 'me': 'e'},
 'BSU00510': {'m': 'e', 'me': 'e'},
 'BSU00810': {'m': None, 'me': '0'},
 'BSU00980': {'m': None, 'me': '0'},
 'BSU00930': {'m': '0', 'me': '0'},
 'BSU00770': {'m': 'e', 'me': 'e'},
 'BSU00790': {'m': 'e', 'me': 'e'},
 'BSU00820': {'m': None, 'me': 'e'},
 'BSU00780': {'m': 'e', 'me': 'e'},
 'BSU01070': {'m': Non

In [34]:
pd.DataFrame.from_dict(GE_dict_ecoli).transpose().to_csv('ecoli_me_essentiality.csv')

In [37]:
A = me.solution.x
B = me.solution.x

In [42]:
C = np.concatenate((A,B),axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [39]:
C

[array([1.09375000e-01, 1.85787375e-02, 6.26391102e-05, ...,
        0.00000000e+00, 0.00000000e+00, 5.28980652e-30]),
 array([1.09375000e-01, 1.85787375e-02, 6.26391102e-05, ...,
        0.00000000e+00, 0.00000000e+00, 5.28980652e-30])]

In [40]:
cobra.

IndexError: list assignment index out of range